# Pilot Notebook: Token Transaction Network Analysis

This Jupyter notebook demonstrates the core functionality of the Token Transaction Network Analyzer, visualizing wallet-to-wallet transaction patterns in token exchange networks. The notebook showcases how to:

1. Load and preprocess transaction data from CSV exports
2. Construct a directed graph representing the transaction network
3. Calculate key metrics like transaction concentration and wallet diversity
4. Generate an interactive visualization highlighting transaction patterns
5. Detect wallets driving significant network activity

The visualization reveals transaction flow dynamics with color-coded nodes representing different wallet behaviors (senders, receivers, concentrated activity) and weighted edges showing transaction frequency between wallets. The accompanying data tables provide detailed metrics on the most active wallets and high-frequency transaction pairs.

This notebook serves as both a demonstration and a starting point for customizing the analysis to your specific token transaction datasets.

# Environment Setup

In [1]:
import polars as pl
import networkx as nx
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Load the CSV data
df = pl.read_csv("/Users/ivan/Desktop/1. Projects/Crypto-Wallet-Network-Activity/dev/test_data/export-token-0x8a16d4bf8a0a716017e8d2262c4ac32927797a2f.csv",
                 truncate_ragged_lines=True)

In [2]:
df

Transaction Hash,Status,Method,BlockNo,DateTime (UTC),From,From_Nametag,To,To_Nametag,Amount,Value (USD)
str,str,str,i64,str,str,str,str,str,f64,str
"""0xde14df2cf61003f6eba54740dfdd…","""Success""","""0x000000ab""",69258893,"""2025-03-19 23:58:33""","""0x2Fef6792255c4A901D11d5a92Bb4…","""""","""0x214b36Cc2Dc66424DA969FCa2494…","""""",0.514805,"""$10.09"""
"""0xde14df2cf61003f6eba54740dfdd…","""Success""","""0x000000ab""",69258893,"""2025-03-19 23:58:33""","""0xa18D1319b944a79d4bCC9F1D0B5D…","""""","""0x2Fef6792255c4A901D11d5a92Bb4…","""""",0.514805,"""$10.09"""
"""0x66c394c67b36df994bd9296c82bf…","""Success""","""0xff040000""",69258871,"""2025-03-19 23:57:47""","""0x5Af622aBce14E8438f665DDc99C6…","""""","""0x424Bb8E613bD2B9dE3432e8423F1…","""""",0.112437,"""$2.20"""
"""0x23612fc7361e83d149a0613393dc…","""Success""","""0x000000ab""",69258781,"""2025-03-19 23:54:35""","""0x2Fef6792255c4A901D11d5a92Bb4…","""""","""0x214b36Cc2Dc66424DA969FCa2494…","""""",0.278605,"""$5.46"""
"""0x23612fc7361e83d149a0613393dc…","""Success""","""0x000000ab""",69258781,"""2025-03-19 23:54:35""","""0xa18D1319b944a79d4bCC9F1D0B5D…","""""","""0x2Fef6792255c4A901D11d5a92Bb4…","""""",0.278605,"""$5.46"""
…,…,…,…,…,…,…,…,…,…,…
"""0xba2749b144f6bcd0f8c4fda7b62e…","""Success""","""Multicall""",69258767,"""2025-03-19 23:54:05""","""0x424Bb8E613bD2B9dE3432e8423F1…","""""","""0x1E191aD29BA045211739FE3BE845…","""""",0.165669,"""$3.25"""
"""0xba2749b144f6bcd0f8c4fda7b62e…","""Success""","""Multicall""",69258767,"""2025-03-19 23:54:05""","""0x6023a9820867e00A9a067B205bB9…","""""","""0x1E191aD29BA045211739FE3BE845…","""""",0.165683,"""$3.25"""
"""0xba2749b144f6bcd0f8c4fda7b62e…","""Success""","""Multicall""",69258767,"""2025-03-19 23:54:05""","""0x6115D0776262110A20fBe190E59A…","""""","""0x1E191aD29BA045211739FE3BE845…","""""",0.165669,"""$3.25"""


# Functions

In [10]:
# Function to clean currency values (assuming you have this defined elsewhere)
def clean_currency(value):
    if isinstance(value, str):
        # Remove currency symbols, commas, etc.
        return value.replace('$', '').replace(',', '')
    return value

# Process each row in the DataFrame using Polars' row-wise operations
def process_row(row):
    from_addr = row['From']
    to_addr = row['To']
    
    # Clean and convert Amount and Value to float
    try:
        amount = float(clean_currency(row['Amount'])) if row['Amount'] is not None else 0
    except (ValueError, TypeError) as e:
        print(f"Warning: Could not convert Amount '{row['Amount']}' to float: {e}")
        amount = 0
        
    try:
        value_usd = float(clean_currency(row['Value (USD)'])) if row['Value (USD)'] is not None else 0
    except (ValueError, TypeError) as e:
        print(f"Warning: Could not convert Value '{row['Value (USD)']}' to float: {e}")
        value_usd = 0
    
    # Add nodes with attributes
    if from_addr not in G:
        G.add_node(from_addr, 
                  label=row['From'],
                  total_sent=0,
                  total_received=0,
                  transaction_count_out=0,
                  transaction_count_in=0)
    
    if to_addr not in G:
        G.add_node(to_addr, 
                  label=row['To'],
                  total_sent=0,
                  total_received=0,
                  transaction_count_out=0,
                  transaction_count_in=0)
    
    # Update node metrics
    G.nodes[from_addr]['total_sent'] += value_usd
    G.nodes[from_addr]['transaction_count_out'] += 1
    G.nodes[to_addr]['total_received'] += value_usd
    G.nodes[to_addr]['transaction_count_in'] += 1
    
    # Add or update edge
    if G.has_edge(from_addr, to_addr):
        G[from_addr][to_addr]['weight'] += 1
        G[from_addr][to_addr]['total_amount'] += amount
        G[from_addr][to_addr]['total_value_usd'] += value_usd
    else:
        G.add_edge(from_addr, to_addr, 
                  weight=1, 
                  total_amount=amount,
                  total_value_usd=value_usd)

# Function to create arrow shapes
def get_arrow(x0, y0, x1, y1, edge_weight, color='rgba(50,50,50,0.5)'):
    # Calculate the direction vector
    dx, dy = x1 - x0, y1 - y0
    # Normalize
    length = np.sqrt(dx**2 + dy**2)
    if length == 0:
        return None
    dx, dy = dx/length, dy/length
    
    # Calculate position for arrowhead (80% along the edge)
    ax, ay = x0 + dx * 0.8 * length, y0 + dy * 0.8 * length
    
    # Calculate perpendicular for arrow wings
    width = 0.03 * (1 + edge_weight/5)
    wx, wy = -dy * width, dx * width
    
    # Points for the arrowhead
    arrow_points = [
        [ax + wx - dx*width*2, ay + wy - dy*width*2],  # Left wing
        [x1, y1],  # Tip
        [ax - wx - dx*width*2, ay - wy - dy*width*2],  # Right wing
    ]
    
    return go.Scatter(
        x=[point[0] for point in arrow_points],
        y=[point[1] for point in arrow_points],
        fill="toself",
        fillcolor=color,
        line=dict(color=color),
        hoverinfo='none',
        mode='lines')


# Data Analysis

In [5]:
# Create a directed graph
G = nx.DiGraph()

# Apply the function to each row in the Polars DataFrame
for row in df.iter_rows(named=True):
    process_row(row)
    
# Calculate network positions - use a layout that better shows directionality
pos = nx.kamada_kawai_layout(G)

# Identify the most active wallets in terms of outgoing transactions
outgoing_counts = {node: G.nodes[node]['transaction_count_out'] for node in G.nodes()}
top_origins = sorted(outgoing_counts.items(), key=lambda x: x[1], reverse=True)[:5]
top_origin_nodes = [node for node, _ in top_origins]

# Create node trace
node_x = []
node_y = []
node_text = []
node_size = []
node_colors = []

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    
    # Node hover text - wallet info
    in_count = G.nodes[node]['transaction_count_in']
    out_count = G.nodes[node]['transaction_count_out']
    received = G.nodes[node]['total_received']
    sent = G.nodes[node]['total_sent']
    
    # Calculate importance metrics
    activity_ratio = out_count / (in_count + 1) # Add 1 to avoid division by zero
    
    hover_text = f"Address: {G.nodes[node]['label']}<br>" + \
                f"Transactions In: {in_count}<br>" + \
                f"Transactions Out: {out_count}<br>" + \
                f"Out/In Ratio: {activity_ratio:.2f}<br>" + \
                f"Total Received: ${received:.2f}<br>" + \
                f"Total Sent: ${sent:.2f}"
    node_text.append(hover_text)
    
    # Size nodes based on transaction volume (with emphasis on outgoing)
    size = 10 + (in_count + out_count * 2)  # Double weight on outgoing to emphasize drivers
    node_size.append(size)
    
    # Color nodes specially if they're top origin nodes
    if node in top_origin_nodes:
        node_colors.append('red')  # Highlight top origins in red
    else:
        # Color by outgoing/incoming ratio
        ratio = out_count / max(in_count, 1)  # Avoid division by zero
        # Blue for mostly receiving, green for balanced, yellow/red for mostly sending
        if ratio < 0.5:
            node_colors.append('rgba(0, 0, 255, 0.8)')  # Blue for receivers
        elif ratio < 1.5:
            node_colors.append('rgba(0, 255, 0, 0.8)')  # Green for balanced
        else:
            node_colors.append('rgba(255, 165, 0, 0.8)')  # Orange for senders

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        size=node_size,
        color=node_colors,
        line_width=2))

# Create edge traces with arrows to show direction
edge_traces = []

# Create curved edge traces to better show directionality
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    
    edge_weight = G[edge[0]][edge[1]]['weight']
    total_value = G[edge[0]][edge[1]]['total_value_usd']
    total_amount = G[edge[0]][edge[1]]['total_amount']
    
    # Width based on transaction count
    width = 1 + (edge_weight / 5)  # Scale edge width
    
    # Color based on source node being a top origin
    if edge[0] in top_origin_nodes:
        color = 'rgba(255, 0, 0, 0.5)'  # Red for edges from top origins
    else:
        color = 'rgba(50, 50, 50, 0.5)'  # Gray for normal edges
    
    # Create the main edge line
    edge_trace = go.Scatter(
        x=[x0, x1],
        y=[y0, y1],
        line=dict(width=width, color=color),
        hoverinfo='text',
        text=f"From: {G.nodes[edge[0]]['label']}<br>" + \
             f"To: {G.nodes[edge[1]]['label']}<br>" + \
             f"Transactions: {edge_weight}<br>" + \
             f"Total Amount: {total_amount}<br>" + \
             f"Total Value: ${total_value:.2f}",
        mode='lines')
    
    edge_traces.append(edge_trace)
    
    # # Add arrowhead
    # arrow = get_arrow(x0, y0, x1, y1, edge_weight, color)
    # if arrow:
    #     edge_traces.append(arrow)

# Calculate centrality metrics to quantify importance
betweenness_centrality = nx.betweenness_centrality(G)
out_degree_centrality = nx.out_degree_centrality(G)

# Identify key driver wallets
driver_scores = {}
for node in G.nodes():
    # Composite score emphasizing outgoing activity
    driver_scores[node] = (
        G.nodes[node]['transaction_count_out'] * 0.5 +
        G.nodes[node]['total_sent'] * 0.3 +
        betweenness_centrality[node] * 100 +
        out_degree_centrality[node] * 100
    )

In [6]:
# Create summary table data with additional metrics
wallet_summary = []
for node in G.nodes():
    in_count = G.nodes[node]['transaction_count_in']
    out_count = G.nodes[node]['transaction_count_out']
    
    wallet_summary.append({
        'Address': G.nodes[node]['label'],
        'TX Out': out_count,
        'TX In': in_count,
        'Out/In Ratio': round(out_count / max(in_count, 1), 2),
        'Total TX': out_count + in_count,
        'Total Sent ($)': round(G.nodes[node]['total_sent'], 2),
        'Total Received ($)': round(G.nodes[node]['total_received'], 2),
        'Driver Score': round(driver_scores[node], 2)
    })

# Convert list of dictionaries to Polars DataFrame
wallet_df = pl.DataFrame(wallet_summary)
# Sort by driver score to emphasize key wallets driving network activity
wallet_df = wallet_df.sort("Driver Score", descending=True).head(10)

# Add a second table for network-wide statistics
stats_df = pl.DataFrame([
    {"Metric": "Total Transactions", "Value": df.height},
    {"Metric": "Total Unique Wallets", "Value": len(G.nodes())},
    {"Metric": "Top Origin Wallet", "Value": G.nodes[top_origin_nodes[0]]['label'] if top_origin_nodes else "None"},
    {"Metric": "Most Active Origin TX Count", "Value": top_origins[0][1] if top_origins else 0},
    {"Metric": "Network Density", "Value": round(nx.density(G), 4)},
])

In [7]:
wallet_df

Address,TX Out,TX In,Out/In Ratio,Total TX,Total Sent ($),Total Received ($),Driver Score
str,i64,i64,f64,i64,f64,f64,f64
"""0xa18D1319b944a79d4bCC9F1D0B5D…",3,0,3.0,3,18.79,0.0,17.14
"""0x5Af622aBce14E8438f665DDc99C6…",2,0,2.0,2,5.45,0.0,12.63
"""0x6023a9820867e00A9a067B205bB9…",2,0,2.0,2,4.55,0.0,12.37
"""0x2Fef6792255c4A901D11d5a92Bb4…",2,2,1.0,4,15.55,15.55,10.93
"""0x214b36Cc2Dc66424DA969FCa2494…",1,2,0.5,3,3.25,15.55,6.74
"""0x36f1959fbAB05CAAEEBC4f70a5d0…",1,1,1.0,2,3.19,1.3,6.72
"""0x424Bb8E613bD2B9dE3432e8423F1…",1,1,1.0,2,3.25,2.2,6.47
"""0x7B41801B47f8279C3A9fAe6Aa61E…",1,0,1.0,1,3.25,0.0,6.47
"""0xb89DaEC63973eEA96bB69979db77…",1,0,1.0,1,3.25,0.0,6.47


# Data Visualization

In [8]:
# Set a modern color palette
colors = {
    'background': '#212529',
    'text': '#f8f9fa',
    'grid': '#495057',
    'origin_node': '#ff6b6b',
    'sender_node': '#ffd166',
    'balanced_node': '#06d6a0',
    'receiver_node': '#118ab2',
    'table_header': '#073b4c',
    'table_cell': '#264653',
    'table_text': '#ffffff',
    'edge': '#64748b',
    'annotation_text': '#ffffff',
    'button': '#0d6efd',
    'button_text': '#ffffff'
}

# Create a darker version of the table header color for the dropdown
dropdown_color = '#052938'  # Darker version of the table header blue

# Create functions to generate node traces with different size metrics
def create_node_trace(nodes, pos, size_by='Driver Score'):
    # Get the appropriate node attribute for sizing
    if size_by == 'Driver Score':
        values = [driver_scores.get(node, 0) for node in nodes]
    elif size_by == 'TX Out':
        values = [G.nodes[node]['transaction_count_out'] for node in nodes]
    elif size_by == 'TX In':
        values = [G.nodes[node]['transaction_count_in'] for node in nodes]
    elif size_by == 'Total Sent':
        values = [G.nodes[node]['total_sent'] for node in nodes]
    elif size_by == 'Total Received':
        values = [G.nodes[node]['total_received'] for node in nodes]
    
    # Normalize sizes relative to maximum (if there's a non-zero max)
    max_value = max(values) if max(values) > 0 else 1
    # Scale between 5 (minimum) and 40 (maximum) for visibility
    sizes = [5 + (35 * (value / max_value)) for value in values]
    
    # Create node colors based on out/in ratio (same logic as before)
    colors = []
    for node in nodes:
        in_count = G.nodes[node]['transaction_count_in']
        out_count = G.nodes[node]['transaction_count_out']
        ratio = out_count / max(in_count, 1)
        
        if node in top_origin_nodes:
            colors.append('#ff6b6b')  # Top origin nodes in red
        elif ratio > 1.5:
            colors.append('#ffd166')  # Sender wallets in orange
        elif 0.5 < ratio <= 1.5:
            colors.append('#06d6a0')  # Balanced wallets in green
        else:
            colors.append('#118ab2')  # Receiver wallets in blue
    
    # Create the updated node trace
    node_trace = go.Scatter(
        x=[pos[node][0] for node in nodes],
        y=[pos[node][1] for node in nodes],
        mode='markers',
        marker=dict(
            size=sizes,
            color=colors,
            line=dict(width=1, color='rgba(250,250,250,0.3)')
        ),
        text=[f"Address: {node}<br>TX Out: {G.nodes[node]['transaction_count_out']}<br>TX In: {G.nodes[node]['transaction_count_in']}<br>Total Sent: ${G.nodes[node]['total_sent']:.2f}<br>Total Received: ${G.nodes[node]['total_received']:.2f}<br>Driver Score: {driver_scores.get(node, 0):.2f}" for node in nodes],
        hoverinfo='text',
        showlegend=False
    )
    
    return node_trace

# Store all the different node trace versions
node_trace_options = {
    'Driver Score': create_node_trace(G.nodes(), pos, 'Driver Score'),
    'TX Out': create_node_trace(G.nodes(), pos, 'TX Out'),
    'TX In': create_node_trace(G.nodes(), pos, 'TX In'),
    'Total Sent': create_node_trace(G.nodes(), pos, 'Total Sent'),
    'Total Received': create_node_trace(G.nodes(), pos, 'Total Received')
}

# Create the subplot figure (graph + tables)
fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.7, 0.3],
    row_heights=[0.8, 0.2],
    specs=[
        [{"type": "scatter", "rowspan": 2}, {"type": "table"}],
        [None, {"type": "table"}]
    ],
    subplot_titles=(
        f"<span style='color:{colors['text']}; font-size:16px'>Token Transaction Network</span>", 
        f"<span style='color:{colors['text']}; font-size:16px'>Top Driver Wallets</span>", 
        f"<span style='color:{colors['text']}; font-size:16px'>Network Statistics</span>"
    )
)

# Add all edge traces with improved styling and hide from legend
for edge_trace in edge_traces:
    # Customize edge traces
    edge_trace.line.color = colors['edge']
    edge_trace.line.width = 1.5
    edge_trace.opacity = 0.7
    edge_trace.showlegend = False
    fig.add_trace(edge_trace, row=1, col=1)

# Add ALL node traces, but make only the default one visible
for idx, (metric, trace) in enumerate(node_trace_options.items()):
    trace.visible = (metric == 'Driver Score')  # Only default is visible
    fig.add_trace(trace, row=1, col=1)

# Add wallet summary table with improved styling
fig.add_trace(
    go.Table(
        header=dict(
            values=list(wallet_df.columns),
            fill_color=colors['table_header'],
            font=dict(color=colors['table_text'], size=13),
            align='left',
            line_color='rgba(255,255,255,0.2)',
            height=30
        ),
        cells=dict(
            values=[wallet_df.get_column(col).to_list() for col in wallet_df.columns],
            fill_color=colors['table_cell'],
            font=dict(color=colors['table_text'], size=12),
            align='left',
            line_color='rgba(255,255,255,0.1)',
            height=25
        ),
        columnwidth=[300, 40, 40, 50, 50, 80, 80, 60]  # Wider Address column
    ),
    row=1, col=2
)

# Add network stats table with improved styling
fig.add_trace(
    go.Table(
        header=dict(
            values=list(stats_df.columns),
            fill_color=colors['table_header'],
            font=dict(color=colors['table_text'], size=13),
            align='left',
            line_color='rgba(255,255,255,0.2)',
            height=30
        ),
        cells=dict(
            values=[stats_df.get_column(col).to_list() for col in stats_df.columns],
            fill_color=colors['table_cell'],
            font=dict(color=colors['table_text'], size=12),
            align='left',
            line_color='rgba(255,255,255,0.1)',
            height=25
        ),
        columnwidth=[150, 100]
    ),
    row=2, col=2
)

# Add only the 4 wallet types to the legend
legend_entries = [
    go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=15, color=colors['origin_node']),
        name='Top Origin Wallets',
        showlegend=True,
        legendgroup="wallet_types"
    ),
    go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=15, color=colors['sender_node']),
        name='Sender Wallets',
        showlegend=True,
        legendgroup="wallet_types"
    ),
    go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=15, color=colors['balanced_node']),
        name='Balanced Wallets',
        showlegend=True,
        legendgroup="wallet_types"
    ),
    go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=15, color=colors['receiver_node']),
        name='Receiver Wallets',
        showlegend=True,
        legendgroup="wallet_types"
    )
]

# Add the legend entries
for entry in legend_entries:
    fig.add_trace(entry, row=1, col=1)

# Calculate the edge trace count and node trace count for button functionality
edge_trace_count = len(edge_traces)
node_trace_count = len(node_trace_options)

# Create updatemenus
updatemenus = [
    dict(
        buttons=[
            dict(
                args=[{
                    'visible': [
                        # Always show edges
                        True if i < edge_trace_count else
                        # Only show the selected node trace
                        (i == edge_trace_count + idx) if edge_trace_count <= i < edge_trace_count + node_trace_count else
                        # Always show tables and legend
                        True
                        for i in range(len(fig.data))
                    ]
                }],
                label=option,
                method='update'
            ) for idx, option in enumerate(['Driver Score', 'TX Out', 'TX In', 'Total Sent', 'Total Received'])
        ],
        direction='down',
        pad={'r': 10, 't': 10},
        showactive=True,
        active=0,  # Index of initially active button
        x=0.1,
        xanchor='left',
        y=1.15,
        yanchor='top',
        # Better dropdown styling for contrast - removed invalid properties
        bgcolor=dropdown_color,  # Darker blue that creates better contrast with white hover
        bordercolor='rgba(255,255,255,0.3)',
        borderwidth=1,
        font=dict(color='rgba(150,150,150,0.9)', size=13)
    )
]

# Update layout with improved styling and buttons
fig.update_layout(
    title=dict(
        text="Token Transaction Network Analysis - Identifying Driver Wallets",
        font=dict(family="Arial, sans-serif", size=24, color=colors['text']),
        x=0.5,
        y=0.98
    ),
    showlegend=True,
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(0,0,0,0.5)',
        bordercolor='rgba(255,255,255,0.2)',
        borderwidth=1,
        font=dict(color=colors['text']),
        itemsizing='constant'
    ),
    updatemenus=updatemenus,
    annotations=[
        dict(text="Size Nodes By:", 
             x=0.055, 
             y=1.105, 
             xref="paper", 
             yref="paper",
             showarrow=False, 
             font=dict(size=14, color=colors['text'], family="Arial, sans-serif"),
             align="left")
    ],
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=100),  # Increased top margin for buttons
    height=950,  # Slightly increased height for buttons
    width=1600,  # Increased width for the table
    paper_bgcolor=colors['background'],
    plot_bgcolor=colors['background'],
    font=dict(family="Arial, sans-serif", color=colors['text']),
    transition_duration=500
)

# Set axis properties
fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False, row=1, col=1)
fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False, row=1, col=1)

# Save and show
fig.write_html("token_transaction_driver_analysis.html", include_plotlyjs='cdn')
fig.show(config={'responsive': True})

## Figure 
Each wallet type in the visualization represents a distinct role in the token transaction network:

- **Top Origin Wallets** - These are the most influential wallets that initiate a high volume of transactions. They act as major sources of token distribution within the network. These wallets typically have a high "out" to "in" transaction ratio and high driver scores, indicating they significantly influence token movement throughout the network.
- **Sender Wallets** - These wallets primarily send tokens to other addresses rather than receiving them. They have a positive out/in ratio (greater than 1) but are not as influential as Top Origin Wallets. They function as intermediaries that help distribute tokens across the network.
- **Balanced Wallets** - These wallets have roughly equal sending and receiving activity. Their out/in transaction ratio is close to 1, indicating they participate in the network as both receivers and distributors. They often represent active traders, exchanges, or wallets that serve multiple functions.
- **Receiver Wallets** - These wallets primarily receive tokens rather than sending them. They have a low out/in ratio (less than 1), functioning as token sinks or end-points in transaction chains. They might represent investors holding assets, user wallets, or inactive addresses.


The **Driver Score** is a composite metric that quantifies a wallet's influence within the token transaction network:

- **Definition**: A weighted calculation that identifies wallets initiating significant transaction activity
- **Components**: Combines outgoing transaction count, total value sent, and network centrality metrics
- **Purpose**: Identifies wallets that play a central role in driving network activity and fund flows
- **Scale**: Higher scores indicate wallets with more network influence or control
- **Calculation**: Weights outgoing transactions (50%), total value sent (30%), and network position (20%)
- **Visualization**: Wallets with high Driver Scores appear larger and more prominent in the network visualization
- **Analysis Value**: Helps identify potential market movers, token redistributors, or coordinating entities
- **Context**: Particularly useful for identifying whale activity or concentrated transaction patterns